## Introduction

In this tutorial, we’ll walk through the process of interacting with a Google Cloud Storage (GCS) bucket named dauphine-bucket, specifically focusing on the data directory within the bucket. We’ll cover how to:

- List all files in the bucket’s data directory.
- Retrieve information about a specific file.
- Read files using the Unstructured library.
- Visualize the extracted documents with LangChain.

This guide is intended for users who are familiar with Python and basic cloud storage concepts.

Prerequisites

Before we begin, ensure you have the following:

- Python 3.x installed on your system.
- Access to the GCP bucket dauphine-bucket/data with the necessary permissions.
- Google Cloud SDK installed and authenticated. You can authenticate by running:

In [4]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=1NoGHgDeQny4R0BgEcJN5ksNvUL5GL&access_type=offline&code_challenge=0wTcop80lOuqktkCoa_z7vUMiXAaJvnRpUA1_z90XDI&code_challenge_method=S256


You are now logged in as [doniatekaya04@gmail.com].
Your current project is [dauphine-437611].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


The following Python libraries installed:
- google-cloud-storage
- unstructured
- langchain

To know more about the libraries, you can visit the following links:
- [google-cloud-storage](https://googleapis.dev/python/storage/latest/index.html)
- [unstructured](https://docs.unstructured.io/examplecode/codesamples/oss/vector-database)
- [langchain](https://langchain.readthedocs.io/en/latest/)


In [15]:
%pip install -q google-cloud-storage unstructured langchain python-magic sqlalchemy langchain_google_cloud_sql_pg
%pip install -q "unstructured[pptx]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# I. Listing and loading files from a GCS bucket

### I.1. Listing Files in the GCP Bucket

Explanation:

To interact with a GCS bucket, we’ll use the google-cloud-storage library. We’ll initialize a client, access the bucket, and list all the files within the data directory.

Code:

In [16]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=tK4SPYP4Qyivb1HaJWWzVExbmfnHck&access_type=offline&code_challenge=YmlGLhS5jh_FNcqAUE748KmYLoUKlUzoOk6xN7dBopw&code_challenge_method=S256


Credentials saved to file: [C:\Users\HP\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "dauphine-437611" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [17]:
# Import the necessary library
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Access the bucket
bucket_name = "dauphine-bucket"
bucket = client.get_bucket(bucket_name)

# List all files in the 'data' directory
blobs = bucket.list_blobs(prefix="data/")

print("Files in 'dauphine-bucket/data':")
for blob in blobs:
    print(blob.name)

Files in 'dauphine-bucket/data':
data/
data/1 - Gen AI - Dauphine Tunis.pptx
data/2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx
data/2.2  - Transformers - Gen AI - Dauphine Tunis.pptx
data/3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx


Output Explanation:

Running this code will display all the file paths within the data directory of the bucket. The prefix='data/' parameter ensures we only get files from that specific directory.

### I.2. Getting Information About One File


Explanation:

Sometimes, you may need detailed information about a specific file, such as its size, content type, or the last time it was updated. We’ll retrieve this metadata for a chosen file.


In [18]:
# Specify the file path (replace with an actual file from your bucket)
file_path = 'data/1 - Gen AI - Dauphine Tunis.pptx'

# Get the blob object
blob = bucket.get_blob(file_path)

# TODO
if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    print(f"Updated On: # TODO")
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")

Information for 'data/1 - Gen AI - Dauphine Tunis.pptx':
Size: 6724048 bytes
Content Type: application/vnd.openxmlformats-officedocument.presentationml.presentation
Updated On: # TODO
Blob name: data/1 - Gen AI - Dauphine Tunis.pptx


Output Explanation:

This code will output metadata about the specified file. Make sure to replace 'data/your_file.ext' with the actual file path.

### I.3. Reading Files with Unstructured

Explanation:

The Unstructured library allows us to parse and process unstructured data from various file formats. We’ll download a file from the bucket and use Unstructured to read and extract its content.

In [16]:
!pip install langchain
!pip install langchain-unstructured


  Using cached langchain_unstructured-0.1.6-py3-none-any.whl.metadata (3.3 kB)
  Using cached onnxruntime-1.19.2-cp311-cp311-win_amd64.whl.metadata (4.7 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.5.4-py3-none-any.whl.metadata (4.7 kB)
Using cached langchain_unstructured-0.1.6-py3-none-any.whl (7.0 kB)
Using cached onnxruntime-1.19.2-cp311-cp311-win_amd64.whl (11.1 MB)
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
Using cached pyreadline3-3.5.4-py3-none-any.whl (83 kB)


In [19]:
# Import necessary libraries
from langchain_core.documents import Document
import os
from google.cloud.storage.bucket import Bucket
from langchain_unstructured import UnstructuredLoader


DOWNLOADED_LOCAL_DIRECTORY = "./downloaded_files"


# Function to download the file: file_path from the GCS Bucket
def download_file_from_bucket(bucket: Bucket, file_path: str) -> str:
    # Download the file locally
    blob = bucket.blob(file_path)

    # data/1 - Gen AI - Dauphine Tunis.pptx' -> 1 - Gen AI - Dauphine Tunis.pptx'
    local_file_name = os.path.basename(file_path)

    # ./downloaded_files/1 - Gen AI - Dauphine Tunis.pptx
    local_filepath = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, local_file_name)

    blob.download_to_filename(local_filepath)
    print(f"Downloaded '{file_path}' to '{local_file_name}'")
    return local_filepath


def read_file_from_local(local_filepath: str) -> list[Document]:
    # Use Unstructured to read the file
    loader = UnstructuredLoader(local_filepath)
    documents = loader.lazy_load()
    return documents

In [24]:
!pip install python-pptx

!pip install python-magic-bin
!pip install python-magic



  Using cached python_magic_bin-0.4.14-py2.py3-none-win_amd64.whl.metadata (710 bytes)
Using cached python_magic_bin-0.4.14-py2.py3-none-win_amd64.whl (409 kB)


In [25]:
# Load all the
blobs = list(bucket.list_blobs(prefix='data/'))
documents: list[Document] = []
if blobs:
    for blob in blobs:
        try:
            local_filepath = download_file_from_bucket(bucket, blob.name)
            documents.extend(read_file_from_local(local_filepath))
        except Exception as e:
            print(f"An error occurred while processing '{blob.name}': {e}")
else:
    print("No files found in the 'data' directory.")

An error occurred while processing 'data/': [Errno 2] No such file or directory: './downloaded_files\\'
Downloaded 'data/1 - Gen AI - Dauphine Tunis.pptx' to '1 - Gen AI - Dauphine Tunis.pptx'
Downloaded 'data/2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx' to '2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx'
Downloaded 'data/2.2  - Transformers - Gen AI - Dauphine Tunis.pptx' to '2.2  - Transformers - Gen AI - Dauphine Tunis.pptx'
Downloaded 'data/3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx' to '3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx'


### I.4. Visualizing the First Documents Extracted with LangChain

Explanation:

LangChain is a framework for developing applications powered by language models. We’ll use it to load and visualize the documents extracted from the file.

In [26]:
for doc in documents[:3]:
    print(f"Content:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")

Content:
Generative AI with LLM
Metadata:
{'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-12T19:04:55', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '48f84e1bdca7ae95acb10169474c3135'}

Content:
Florian Bastin
Metadata:
{'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-12T19:04:55', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': 'f4361d35367e66b1a0e01ea107730df7'}

Content:
👨🏼‍🎓 Master MASH - Université PSL
Metadata:
{'source': './downloaded_files\\1 - G

### I.5. Join extracted document by page

Explanation:

- The text extraction block is uninformative because very small text blocks are extracted from the document.
- We can join the extracted text by page to get a more meaningful output.
- A metadata with the 'page_number' can be helpful
- The other metadatas need to be merged

In [53]:
# Merge the documents by page
merged_documents = merge_documents_by_page(documents)

# Check if any documents were merged
if not merged_documents:
    print("No documents were merged.")
else:
    # Print the merged documents
    for doc in merged_documents:
        print("-" * 50)
        print(f"Page Number: {doc.metadata.get('page_number')}")
        print(f"Source: {doc.metadata.get('source')}")
        print(f"Content:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")
        print("-" * 50)


--------------------------------------------------
Page Number: 1
Source: ./downloaded_files\1 - Gen AI - Dauphine Tunis.pptx
Content:
Generative AI with LLM
Florian Bastin
👨🏼‍🎓 Master MASH - Université PSL
👨🏼‍💻 LLM Engineer @OctoTechnology
Le Monde, Casino, Channel, Club Med, Pernod Ricard, Suez
‹#›
Metadata:
{'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-12T19:04:55', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '48f84e1bdca7ae95acb10169474c3135'}

--------------------------------------------------
--------------------------------------------------
Page Number: 2
Source: ./downloaded_files\1 - Gen AI - Dauphine Tunis.pptx
Content:
I.A Pretraining Large Language Model
A. Pretraining a Large Language Model
Introducti

# II. Ingesting in Cloud SQL

We will ingest each merged_document in Cloud SQL.

ALREADY DONE by teacher: 
- Create a Cloud SQL instance
- Create a database in the instance


TODO:
- Create a table in CloudSQL with you initials
- Create the schema of the table
- Ingest the data in the table


Follow this [documentation](https://python.langchain.com/docs/integrations/vectorstores/google_cloud_sql_pg/)

### II.1 Understand how to connect to Cloud SQL 


First we need to connect to Cloud SQL 
- Follow this [link](https://cloud.google.com/sql/docs/postgres/connect-instance-auth-proxy) to understand how it works

Then be familiar ith the following PostgreSQL commands:
```bash 
`psql "host=127.0.0.1 port=5432 sslmode=disable dbname=gen_ai_db user=postgres"` # to connect to the user `postgres`
# the user we use is `students`
# a password provided by the teacher is required
`\l` # to list all databases
`\c gen_ai_db` # to connect to the database `gen_ai_db`
`\dt` # to list all tables
`\d+ table_name` # to describe a table
`SELECT * FROM table_name` # to select all rows from a table
`\du` # to list all users
`\q` # to quit
`CREATE DATABASE db_name;` # to create a database
`CREATE USER user_name WITH PASSWORD 'password';` # to create a user
`GRANT ALL PRIVILEGES ON DATABASE db_name TO user_name;` # to grant all privileges to a user on a database
`GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO user_name;` # to grant all privileges to a user on all tables in a schema
`ALTER USER user_name WITH SUPERUSER;` # to grant superuser privileges to a user
`DROP DATABASE db_name;` # to drop a database
`DROP USER user_name;` # to drop a user
`DROP TABLE table_name;` # to drop a table
`REVOKE ALL PRIVILEGES ON DATABASE db_name FROM user_name;` # to revoke all privileges from a user on a database
```

When Cloud SQL Proxy is downloaded and the tutorial is followed. You should be connected to the instance. 
You can connect to the dabase as a user `students` with the password provided by the teacher.
  - `psql "host=127.0.0.1 port=5432 sslmode=disable dbname=gen_ai_db user=students"`
  - Enter the password provided by the teacher
Try to create a table `initial_tests_table` with the following schema:
  - `CREATE TABLE initial_tests_table (id SERIAL PRIMARY KEY, document TEXT, page_number INT, title TEXT, author TEXT, date TEXT);`
  - `\dt` to check if the table has been created
  - `\d+ initial_tests_table` to check the schema of the table
  - `DROP TABLE initial_tests_table;` to drop the table
  - `\q` to quit


In [82]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
import os
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [44]:
TABLE_NAME = "dt_table" # Table name in the database initials-table. Ex: fb_table

In [46]:
from langchain_google_cloud_sql_pg import PostgresEngine

# Connect to the PostgreSQL database
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD,
)


In [47]:
# Create a table in the PostgreSQL database with the required columns
from sqlalchemy.exc import ProgrammingError

try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME, # Vector size for VertexAI model(textembedding-gecko@latest)
        vector_size=768,
    )
except ProgrammingError:
    print("Table already created")

Table already created


- Execute \d+ [YOUR_INITIALS]_table in the psql shell to check the schema of the table

### II.2 Create an embedding to convert your documents

In [48]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


In [49]:
from langchain_google_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(  # Use .create() to initialize an async vector store
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)

In [57]:
await vector_store.aadd_documents(merged_documents)


['ea7d5a56-8e03-4429-a2a6-cd7c0b29f88f',
 '7e4ae5bf-d8ab-497c-b5ea-87a88eef5926',
 '84885de3-5ed4-4675-b665-e8160432d7d2',
 '3f30d4e6-ecda-497d-bd62-e2e2c08aa69b',
 '1073bcfb-2d2b-4be4-8a2e-9841e303e447',
 '4d5a5047-2984-476f-afc1-857ebab7795c',
 '75e55a75-bdeb-47ab-ba8b-ff733502bbd8',
 '3fa0c503-5eab-4bb2-a95e-c3ebf1de25e8',
 'e24a5968-9127-48cf-9445-14ea45e3ec90',
 '9985cfb8-c543-4236-9224-09f3932ab2e7',
 '17e1b0bc-1652-43b7-9f85-b1bd2fa482be',
 '8328cf97-e584-4643-8541-f5cfdb252de4',
 '8ed9f78d-2370-4906-8902-03dda15e4f7e',
 'fd214a4f-8e0a-4603-bb93-fb5cfa999161',
 'a3ce6559-14e4-47fa-8ee2-7b09fd26541f',
 'f7e2bab6-da9a-4dd0-9ffa-8f454741e72a',
 '575a4400-1f11-4f40-864e-1790c9b0659f',
 '3526c55b-cdf4-4190-9ae6-40045c197ad9',
 'af51890c-7bfb-4235-b875-dc61c8ab6a46',
 'b4f8c8ce-053b-4101-a961-bffe42aae95a',
 'd132efc1-7532-4371-9b18-f1bc9784f6a3',
 'bf470543-91fb-484d-8e45-da89e8caef44',
 '19a47602-ebea-43bf-b29e-18e5c91b597a',
 '7255617b-cf42-43dc-8b33-f2e996b6dfb6',
 '2a639e84-0b97-

### II.3 Perform a similarity search

In [58]:
query = "How to train a Large Language Model?"

In [59]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

docs = retriever.invoke(query)

In [60]:
for doc in docs:
    print("-" * 50)
    print("Content: ", doc.page_content)
    print("Metadata: ", doc.metadata)

--------------------------------------------------
Content:  I.A.7 Training Process
Training process
Steps 
Find scaling recipes (example: learning rate decrease if the size of the model increase)
Tune hyper parameters on small models of differents size
Choose the best models among the smallest ones
Train the biggest model with the 
Q. Should I use Transformers or LSTM ? 
‹#›
Stanford CS229 I Machine Learning I Building Large Language Models (LLMs) [Youtube]
Metadata:  {'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 0, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-12T19:04:55', 'page_number': 14, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '4edca6e6a813ce76686e4f90e2143a4e'}
--------------------------------------------------
Content:  I.B Fine tuning Large Language Model
B. Fine tun

**Congratulations**! You have successfully ingested the data in Cloud SQL.